In [8]:
from PIL import Image, ImageDraw
import os
from zipfile import ZipFile

In [3]:
# Dataset 위치
dataset_path = 'Dataset/'
zip_list = os.listdir(dataset_path)
zip_list

['19062421.zip', '19062431.zip']

In [4]:
# unzip dataset
for z in zip_list:
    if z == '.ipynb_checkpoints':
        continue
    with ZipFile(dataset_path + z, 'r') as zip:
        print(z + ': unzip 시작')
        zip.extractall(dataset_path + z.replace('.zip' ,''))
        print(z + ': unzip 완료')

19062421.zip: unzip 시작
19062421.zip: unzip 완료
19062431.zip: unzip 시작
19062431.zip: unzip 완료


In [6]:
# 전처리한 데이터를 저장할 디렉토리 생성
def createDir(dir_name):
    try:
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
    except OSError:
        print('Error: 디렉터리 생성오류. 디렉토리명: ' + dir_name)

In [5]:
# 사용할 사진 경로
accessory = 'S001'  # 보통, 안경
light = 'L1'
emotion = ('E01', 'E02')      # 무표정, 웃음
camera = ('C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20')

In [60]:
# bbox 그리기
def show_bbox(img_path, bbox):

    img = Image.open(dataset_path + img_path)
    draw = ImageDraw.Draw(img)
    bbox = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]
    draw.rectangle(bbox, outline=(0, 255, 0), width=1)

    img.show()

In [43]:
# bbox 정보 구조화
# 코 끝(0), 우안(1), 좌안(2), 우측 입꼬리(3), 좌측 입꼬리(4), 우측 귀(5), 좌측 귀(6)
def bbox_info(txt_path):
    
    info = {}
    
    with open(dataset_path + txt_path , 'r') as f:
        
        # 좌안, 우안 위치 확인
        for i in range(0, 7):
            coordinate = f.readline()
            if i == 5:
                info['right_ear'] = tuple(map(int, coordinate.replace('\n', '').split('\t')))
            elif i == 6:
                info['left_ear'] = tuple(map(int, coordinate.replace('\n', '').split('\t')))
        
        # 얼굴 bbox
        face_bbox = f.readline()
        info['face_bbox'] = tuple(map(int, face_bbox.replace('\n', '').split('\t')))
        
        while True:
            
            bbox = f.readline()
            
            if bbox == '':
                break
                
    return info

In [75]:
# face_bbox 정보 수정
def resize_bbox(info):
    resized_bbox = list(info['face_bbox'])
    resized_bbox[0] = info['right_ear'][0]   # start_x
    resized_bbox[1] = info['face_bbox'][1]  # start_y
    resized_bbox[2] = info['left_ear'][0]  # start_x + width
    resized_bbox[3] = info['face_bbox'][1] + info['face_bbox'][3]  # start_y + height
    
    return resized_bbox

In [76]:
# test
info = bbox_info('/19062421/S001/L1/E01/C4.txt')
print(info)
resized_bbox = resize_bbox(info)
print(resized_info)
show_bbox('/19062421/S001/L1/E01/C4.jpg', resized_bbox)

{'right_ear': (101, 56), 'left_ear': (121, 54), 'face_bbox': (64, 22, 64, 64)}
[66, 26, 41, 61]


In [77]:
# crop image
processed_path = "ProcessedDataset/"
i = 0
for z in zip_list:
    face_id = z.replace('.zip', '')
    for e in emotion:
        for c in camera:
            path = '/{face_id}/{accessory}/{light}/{emotion}/{camera}'.format(
                face_id=face_id, 
                accessory=accessory,
                light=light,
                emotion=e,
                camera=c
            )
            createDir(processed_path + face_id)
            
            img_path = path + '.jpg'
            txt_path = path + '.txt'
            # bbox에서 양쪽 귀 좌표 추출
            info = bbox_info(txt_path)
            resized_bbox = resize_bbox(info)
            print(resized_bbox)

            # crop
            img = Image.open(dataset_path +img_path)
            print(img.size)
            cropped_img = img.crop(tuple(resized_bbox))
            cropped_img_path = processed_path + face_id +'/' + str(i) + '.jpg'
            print(cropped_img_path)
            cropped_img.save(cropped_img_path)
            i += 1



[101, 22, 121, 86]
(173, 115)
ProcessedDataset/19062421/0.jpg
[89, 19, 114, 79]
(173, 115)
ProcessedDataset/19062421/1.jpg
[77, 24, 109, 82]
(173, 115)
ProcessedDataset/19062421/2.jpg
[72, 26, 105, 83]
(173, 115)
ProcessedDataset/19062421/3.jpg
[67, 28, 94, 83]
(173, 115)
ProcessedDataset/19062421/4.jpg
[61, 26, 88, 80]
(173, 115)
ProcessedDataset/19062421/5.jpg
[58, 25, 87, 81]
(173, 115)
ProcessedDataset/19062421/6.jpg
[55, 24, 72, 83]
(173, 115)
ProcessedDataset/19062421/7.jpg
[81, 18, 111, 74]
(173, 115)
ProcessedDataset/19062421/8.jpg
[74, 25, 105, 78]
(173, 115)
ProcessedDataset/19062421/9.jpg
[69, 18, 96, 73]
(173, 115)
ProcessedDataset/19062421/10.jpg
[61, 19, 86, 75]
(173, 115)
ProcessedDataset/19062421/11.jpg
[84, 30, 115, 84]
(173, 115)
ProcessedDataset/19062421/12.jpg
[64, 30, 90, 82]
(173, 115)
ProcessedDataset/19062421/13.jpg
[101, 24, 122, 82]
(173, 115)
ProcessedDataset/19062421/14.jpg
[89, 21, 117, 80]
(173, 115)
ProcessedDataset/19062421/15.jpg
[81, 25, 111, 82]
(173,